In [1]:
# package imports
import torch
import huggingface
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, GPT2Model, GPT2Tokenizer, AutoModelForCausalLM,GPT2LMHeadModel, GPT2Config)
from datasets import load_dataset

In [2]:
#import validation dataset for evaluation. 
dataset = load_dataset('rotten_tomatoes',split='validation') #just for evaluating

In [3]:
#shape of dataset
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1066
})

In [4]:
import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

C:\Users\felix\Downloads


In [5]:
#instantiate model

#create tokenizer with padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
pad_token = "<PAD>"
tokenizer.pad_token = pad_token
#tokenizer.set_padding(tokenizer.pad_token, pad_to_multiple_of=8)
config = GPT2Config.from_pretrained("gpt2", pad_token_id=tokenizer.pad_token_id)
model = AutoModelForSequenceClassification.from_pretrained("gpt2",config=config)
#config = GPT2Config.from_pretrained("gpt2", pad_token_id=tokenizer.pad_token_id)

# Instantiate the model with the updated configuration
#model = GPT2ForSequenceClassification.from_pretrained("gpt2", config=config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
#create a tokenized dataset for evaluation
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [8]:
#creating input for model
inputs = tokenizer.encode_plus(
    tokenized_datasets['text'],
    add_special_tokens=True,
    max_length=128,  # Maximum sequence length
    padding="max_length",
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)

In [9]:
# Make prediction
with torch.no_grad():
    outputs = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities)

# Display sentiment result
if predicted_class == 1:
    print(f"Sentiment: Positive ({probabilities[0][1] * 100:.2f}%)")
else:
    print(f"Sentiment: Negative ({probabilities[0][0] * 100:.2f}%)")

Sentiment: Negative (99.59%)


In [10]:
#dataset labels
labels = dataset["label"]

In [11]:
#load full dataset for testing
full_dataset = load_dataset('rotten_tomatoes')

In [12]:
# Create a DataLoader to efficiently process the data
data_loader = torch.utils.data.DataLoader(list(zip(inputs["input_ids"],inputs["attention_mask"], labels)),
batch_size=16, shuffle=False)

#evaluate model performance
from sklearn.metrics import accuracy_score

tokenized_dataset = full_dataset.map(lambda examples: tokenizer(examples["text"], padding=True, truncation=True))
                                     #, batched=True)

# Prepare the data for evaluation
eval_dataset = tokenized_dataset["test"].remove_columns(["text"]).rename_column("label", "labels")
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()
predictions = []
for batch in torch.utils.data.DataLoader(eval_dataset):
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())

true_labels = eval_dataset["labels"].numpy()
accuracy = accuracy_score(true_labels, predictions)
print("The pretrained model accuracy is", round(accuracy*100,2),"%")

In [13]:
from peft import LoraConfig
# Create a PEFT Config for LoRA
from peft import LoraConfig, TaskType
config = LoraConfig(
r=8, # Rank
lora_alpha=32,
target_modules=['c_attn', 'c_proj'],
lora_dropout=0.1,
bias="none",
task_type=TaskType.SEQ_CLS
)

In [14]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

C:\Users\felix\anaconda3\Lib\site-packages\peft\tuners\lora\layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [15]:
lora_model.print_trainable_parameters()

trainable params: 812,544 || all params: 125,253,888 || trainable%: 0.6487175871139426


In [16]:
#create a training dataset for PEFT model
new_dataset = full_dataset.map(lambda examples: tokenizer(examples["text"], padding=True, truncation=True))

In [17]:
new_dataset = new_dataset.rename_column("label", "labels")
new_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [18]:
#unfreeze model
for param in lora_model.parameters():
    param.requires_grad = True

In [19]:
import numpy as np
import torch.nn as nn
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return{"accuracy": (predictions == labels).mean()}

In [21]:
#this one gets no training or validation loss
#def compute_metrics(eval_pred):
#    predictions = eval_pred.predictions
#    labels = eval_pred.label_ids

    # Convert predictions and labels to the appropriate data types
#    predictions = torch.from_numpy(predictions).float()
#    labels = torch.from_numpy(labels).long()

#    loss = nn.CrossEntropyLoss()
#    loss_val = loss(predictions, labels)
#    accuracy = (predictions.argmax(dim=1) == labels).float().mean()

#    return {"eval_loss": loss_val.item(), "accuracy": accuracy.item()}

In [22]:
#no logs and returns eval_loss error
#def compute_metrics(eval_pred):
#    predictions = eval_pred.predictions
#    labels = eval_pred.label_ids

    #Convert predictions and labels to the appropriate data types
#    predictions = torch.from_numpy(predictions).float()
#    labels = torch.from_numpy(labels).long()

#    loss = nn.CrossEntropyLoss()
#    loss_val = loss(predictions, labels)
#    accuracy = (predictions.argmax(dim=1) == labels).float().mean()

#    return {"eval_loss": loss_val, "accuracy": accuracy}


In [23]:
#error and no log 
#def compute_metrics(eval_pred):
#    predictions = torch.from_numpy(eval_pred.predictions)  # Convert predictions to tensor
#    labels = torch.from_numpy(eval_pred.label_ids).float()  # Convert labels to tensor and cast to float
#    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
#    accuracy = (predictions == labels).float().mean()  # Calculate the accuracy
#    return {"eval_loss": loss, "accuracy": accuracy}

In [24]:
#def compute_metrics(eval_pred):
#    predictions, labels = eval_pred
#    predictions = np.argmax(predictions, axis=1)
#    predictions = torch.from_numpy(predictions)  # Convert predictions to tensor
#    labels = torch.from_numpy(labels).float()  # Convert labels to tensor
#    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
#    accuracy = (predictions == labels).float().mean()  # Calculate the accuracy
#    return {"eval_loss": loss, "accuracy": accuracy}


In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    predictions = torch.from_numpy(predictions)  # Convert predictions to tensor
    labels = torch.from_numpy(labels).float()  # Convert labels to tensor
    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
    accuracy = (predictions == labels).float().mean()  # Calculate the accuracy
    return {"eval_loss": loss.item(), "accuracy": accuracy.item()}

In [26]:
training_args = TrainingArguments(
    output_dir='C:/Users/felix/Downloads',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=5,
    load_best_model_at_end=True,
    #weight_decay=0.1,
    remove_unused_columns=False,
    #label_names="labels"
)

In [27]:
import random

In [28]:
# reduce size of training data to speed up
n_samples =800
train_dataset = new_dataset['train']

# Get the number of samples in the dataset
num_samples = len(train_dataset)

# Generate a list of random indices without replacement
random_indices = random.sample(range(num_samples), n_samples)

# Select the samples corresponding to the random indices
random_train_samples = train_dataset.select(indices=random_indices)

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.from_numpy(predictions)  # Convert predictions to tensor
    labels = torch.from_numpy(labels).long()  # Convert labels to tensor
    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
    accuracy = (torch.argmax(predictions, axis=1) == labels).float().mean()  # Calculate the accuracy

    # Print the metrics dictionary for debugging
    metrics = {"eval_loss": loss.item(), "accuracy": accuracy.item()}
    print("Metrics:", metrics)

    return metrics

In [30]:
trainer = Trainer(
    model=lora_model,
    args = training_args,
    train_dataset = random_train_samples,
    eval_dataset = new_dataset['test'],
    tokenizer=tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics = compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.717404,0.500000
2,No log,0.692329,0.500938
3,No log,0.709676,0.509381
4,No log,0.694390,0.500000
5,No log,0.693648,0.503752


Metrics: {'eval_loss': 0.7174040675163269, 'accuracy': 0.5}
Metrics: {'eval_loss': 0.6923287510871887, 'accuracy': 0.5009380578994751}
Metrics: {'eval_loss': 0.709675669670105, 'accuracy': 0.5093808770179749}
Metrics: {'eval_loss': 0.6943901777267456, 'accuracy': 0.5}
Metrics: {'eval_loss': 0.6936475038528442, 'accuracy': 0.5037523508071899}


TrainOutput(global_step=335, training_loss=0.7597835711578824, metrics={'train_runtime': 2145.5782, 'train_samples_per_second': 1.864, 'train_steps_per_second': 0.156, 'total_flos': 92308962078720.0, 'train_loss': 0.7597835711578824, 'epoch': 5.0})

In [31]:
trainer.evaluate()

Metrics: {'eval_loss': 0.6937015056610107, 'accuracy': 0.5037523508071899}


{'eval_loss': 0.693701446056366,
 'eval_accuracy': 0.5037523508071899,
 'eval_runtime': 100.6148,
 'eval_samples_per_second': 10.595,
 'eval_steps_per_second': 0.885,
 'epoch': 5.0}